In [ ]:
import sys
sys.path.append('../../')

In [ ]:
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from vqvae_celeba import VQVAE
import numpy as np
import matplotlib.pyplot as plt
from torchvision.utils import make_grid

In [ ]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.CenterCrop(148),
    transforms.Resize(64),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

train_dataset = datasets.CelebA(root='../../data', split='train', transform=transform, download=False)
val_dataset = datasets.CelebA(root='../../data', split='test', transform=transform, download=False)

train_loader = DataLoader(train_dataset,
                            batch_size=32,
                            num_workers=8,
                            shuffle=True,
                            drop_last=True)

train_loader = DataLoader(
    train_dataset,
    batch_size=256,
    shuffle=True,
    num_workers=8,
    pin_memory=True,
)
val_loader = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=8,
    pin_memory=True,
)

In [ ]:
model = VQVAE.load_from_checkpoint(
    '../../lightning_logs/vqvae_celeba/version_0/checkpoints/epoch=17-step=11447.ckpt',
    num_hiddens=128,
    num_residual_hiddens=32,
    num_residual_layers=2,
    num_embeddings=512,
    embedding_dim=64,
    commitment_cost=0.25)

## Reconstruction

In [ ]:
valid_originals, _ = next(iter(val_loader))
_, valid_recons, _ = model(valid_originals)

In [ ]:
valid_recons.shape

In [ ]:
def show(img):
    npimg = img.numpy()
    fig = plt.figure(figsize=(10, 20))
    plt.imshow(np.transpose(npimg, (1, 2, 0)), interpolation='nearest')
    plt.axis('off')

In [ ]:
show(make_grid(valid_recons.cpu(), normalize=True))

In [ ]:
show(make_grid(valid_originals.cpu(), normalize=True))